# Natural Language Processing

Implement a text classification model using a dummy text dataset. First create that dummy dataset inline, then train a text classificaiton model. Evaluate model performance by metrics such as Accuracy, F1-score and save the model to textClassification.mlnet, Once completed, print 'task resolved' in the end.

In [ ]:
#r "nuget:Microsoft.ML"
#r "nuget:Microsoft.ML.Data"


In [ ]:
using System;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;

// Define data structure
public class TextData
{
public string Text { get; set; }
public string Label { get; set; }
}

public class TextPrediction
{
[ColumnName("PredictedLabel")]
public string PredictedLabel;
}

var context = new MLContext();

// Create dummy data
var data = context.Data.LoadFromEnumerable(new TextData[]
{
new TextData { Text = "I love this game", Label = "Positive" },
new TextData { Text = "This is a bad movie", Label = "Negative" },
new TextData { Text = "I enjoy this music", Label = "Positive" },
new TextData { Text = "This is not a good book", Label = "Negative" },
});

// Split data into training and testing
var tt = context.Data.TrainTestSplit(data);

// Define pipeline
var pipeline = context.Transforms.Text.FeaturizeText("Features", "Text")
.Append(context.Transforms.Conversion.MapValueToKey("Label"))
.Append(context.Transforms.Concatenate("Features", "Features"))
.Append(context.MulticlassClassification.Trainers.SdcaNonCalibrated())
.Append(context.Transforms.Conversion.MapKeyToValue("PredictedLabel"));

// Train model
var model = pipeline.Fit(tt.TrainSet);

// Evaluate model
var predictions = model.Transform(tt.TestSet);
var metrics = context.MulticlassClassification.Evaluate(predictions);
Console.WriteLine($"Accuracy: {metrics.MacroAccuracy}");
Console.WriteLine($"Log-loss: {metrics.LogLoss}");
Console.WriteLine($"Per-class Log-loss: {String.Join(", ", metrics.PerClassLogLoss.Select((pl, i) => $"Class {i}: {pl}"))}");

// Save model
context.Model.Save(model, tt.TrainSet.Schema, "textClassification.mlnet");

Console.WriteLine("task resolved");